In [1]:
import re
import time
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup

from datetime import timedelta
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta

In [2]:
class Get_amedas_station:
  def __init__(self):
    url = 'https://www.data.jma.go.jp/obd/stats/etrn/select/prefecture00.php?prec_no=&block_no=&year=&month=&day=&view='
    html = urllib.request.urlopen(url)
    self.soup = BeautifulSoup(html, 'html.parser')

  def get_area_link(self):
    elements = self.soup.find_all('area')
    self.area_list = [element['alt'] for element in elements]
    self.area_link_list = [element['href'] for element in elements]

  def get_station_link(self):
    out = pd.DataFrame(columns=['station','url','area'])
    for area, area_link in zip(self.area_list, self.area_link_list):
      url = 'https://www.data.jma.go.jp/obd/stats/etrn/select/'+ area_link
      html = urllib.request.urlopen(url)
      soup = BeautifulSoup(html, 'html.parser')
      elements = soup.find_all('area')
      station_list = [element['alt'] for element in elements]
      station_link_list = [element['href'].strip('../') for element in elements]
      df1 = pd.DataFrame(station_list,columns=['station'])
      df2 = pd.DataFrame(station_link_list,columns=['url'])
      df = pd.concat([df1, df2],axis=1).assign(area=area)
      out = pd.concat([out,df])
      print(area)
    self.out = out

  def data_arange(self):
    out = self.out[~self.out.duplicated()].assign(append='https://www.data.jma.go.jp/obd/stats/etrn/')
    out['amedas_url'] = out['append'] + out['url']
    out = out.loc[:,['area','station','amedas_url']]
    out.to_csv('amedas_url_list.csv',index=None, encoding='SJIS')

## 次の関数で使うURIのリスト(amedas_url_list.csv)を作成する関数の実行

In [ ]:
amedas = Get_amedas_station()
amedas.get_area_link()
amedas.get_station_link()
amedas.data_arange()

## 気象庁から指定した条件でcsvを出力する関数

In [4]:
# 各AMeDAS地点のurlから取得対象データを検索し、リスト化するメソッド
def search_data(url):
  html = urllib.request.urlopen(url)
  time.sleep(1)
  soup = BeautifulSoup(html, 'html.parser')
  element = soup.find_all('tr', attrs={'class':'mtx', 'style':'text-align:right;'})
  out = [list(map(lambda x: x.text, ele)) for ele in element]
  return out

class Get_amedas_data:
  def __init__(self,area_name,station_name):
    # 指定した地域と地点から対象のurlを参照する
    self.st_name = station_name
    self.a_name = area_name
    amedas_url_list = pd.read_csv('amedas_url_list.csv',encoding='SJIS')
    df = amedas_url_list[(amedas_url_list['area']==self.a_name) & (amedas_url_list['station']==self.st_name)]
    amedas_url = df.iat[0,2]
    # urlから正規表現で数字を取得する（prec_noとblock_noのidが必要）
    pattern=r'([+-]?[0-9]+\.?[0-9]*)'
    id_list=re.findall(pattern, amedas_url)
    self.pre_id = id_list[0]
    self.s_id = id_list[1]

  # 1日データは1月分のデータとしてまとめて取得できるため、開始月と終了月を指定する
  def set_date1(self, startmonth, endmonth):
    self.start = startmonth
    self.end = endmonth
    strdt = dt.strptime(self.start, '%Y%m')
    enddt = dt.strptime(self.end, '%Y%m')
    months_num = (enddt.year - strdt.year)*12 + enddt.month - strdt.month + 1
    # 開始月〜終了月までの月をリスト化
    self.datelist = map(lambda x, y=strdt: y + relativedelta(months=x), range(months_num))

  # 1時間 or 10分データに関しては、開始日と終了日を指定する
  def set_date2(self,startdate,enddate):
    self.start = startdate
    self.end = enddate
    strdt = dt.strptime(self.start, '%Y%m%d')
    enddt = dt.strptime(self.end, '%Y%m%d')
    days_num = (enddt - strdt).days + 1
    # 開始日〜終了日までの日付をリスト化
    self.datelist = map(lambda x, y=strdt: y + timedelta(days=x), range(days_num))

  # 予め空のデータフレームを作成しておく。気象台のある地点の方が取得できる要素が多い。
  # 取得したい時間スケールを指定（type）
  def dl_data(self, type):
    # dailyデータに関する空のデータフレーム
    data1  = pd.DataFrame(columns=['年月','日','平均現地気圧','平均海面気圧','日降水量','最大1時間降水量','最大10分間降水量','平均気温','最高気温','最低気温','平均湿度','最小湿度','平均風速','最大風速','最大風向','最大瞬間風速','最大瞬間風向','日照時間','降雪','最深積雪','天気概況（昼）','天気概況（夜）'])
    data1_ = pd.DataFrame(columns=['年月','日','日降水量','最大1時間降水量','最大10分間降水量','平均気温','最高気温','最低気温','平均風速','最大風速','最大風向','最大瞬間風速','最大瞬間風向','最多風向','日照時間','降雪','最深積雪'])
    # hourlyデータに関する空のデータフレーム
    data2  = pd.DataFrame(columns=['日付','時','現地気圧','海面気圧','降水量','気温','露点温度','蒸気圧','湿度','風速','風向','日照時間','全天日射量','降雪','積雪','天気','雲量','視程'])
    data2_ = pd.DataFrame(columns=['日付','時','降水量','気温','風速','風向','日照時間','降雪','積雪'])
    # 10minデータに関する空のデータフレーム
    data3  = pd.DataFrame(columns=['日付','時分','現地気圧','海面気圧','降水量','気温','相対湿度','平均風速','平均風向','最大瞬間風速','最大瞬間風向','日照時間'])
    data3_ = pd.DataFrame(columns=['日付','時分','降水量','気温','平均風速','平均風向','最大瞬間風速','最大瞬間風向','日照時間'])

    # リスト化した月 or 日付リストを回しながらデータを取得しつつ、縦に結合しながらデータフレームを作成する
    for dt in self.datelist:
      d = dt.strftime("%Y%m%d")
      yyyy = d[0:4]
      mm = d[4:6]
      dd = d[6:8]

      if type=='daily':
        # 気象台のある地点のblock_noは5桁の番号
        if len(self.s_id) == 5:
          pattern = 's1'
          url = f'https://www.data.jma.go.jp/obd/stats/etrn/view/{type}_{pattern}.php?prec_no={self.pre_id}&block_no={self.s_id}&year={yyyy}&month={mm}&day={dd}&view=p1'
          out = search_data(url)
          df = (pd.DataFrame(out, columns=['日','平均現地気圧','平均海面気圧','日降水量','最大1時間降水量','最大10分間降水量','平均気温','最高気温','最低気温','平均湿度','最小湿度','平均風速','最大風速','最大風向','最大瞬間風速','最大瞬間風向','日照時間','降雪','最深積雪','天気概況（昼）','天気概況（夜）'])).assign(年月=f'{yyyy}{mm}')
          df = df.loc[:,['年月','日','平均現地気圧','平均海面気圧','日降水量','最大1時間降水量','最大10分間降水量','平均気温','最高気温','最低気温','平均湿度','最小湿度','平均風速','最大風速','最大風向','最大瞬間風速','最大瞬間風向','日照時間','降雪','最深積雪','天気概況（昼）','天気概況（夜）']]
          data1 = pd.concat([data1, df])
          data1.to_csv(f'{self.a_name}_{self.st_name}_{self.start}-{self.end}_{type}.csv',index=None, encoding='SJIS')
        else:
          pattern = 'a1'
          url = f'https://www.data.jma.go.jp/obd/stats/etrn/view/{type}_{pattern}.php?prec_no={self.pre_id}&block_no={self.s_id}&year={yyyy}&month={mm}&day={dd}&view=p1'
          out = search_data(url)
          df = (pd.DataFrame(out, columns=['日','日降水量','最大1時間降水量','最大10分間降水量','平均気温','最高気温','最低気温','平均風速','最大風速','最大風向','最大瞬間風速','最大瞬間風向','最多風向','日照時間','降雪','最深積雪'])).assign(年月=f'{yyyy}{mm}')
          df = df.loc[:,['年月','日','日降水量','最大1時間降水量','最大10分間降水量','平均気温','最高気温','最低気温','平均風速','最大風速','最大風向','最大瞬間風速','最大瞬間風向','最多風向','日照時間','降雪','最深積雪']]
          data1_ = pd.concat([data1_, df])
          data1_.to_csv(f'{self.a_name}_{self.st_name}_{self.start}-{self.end}_{type}.csv',index=None, encoding='SJIS')

      elif type=='hourly':
        if len(self.s_id) == 5:
          pattern = 's1'
          url = f'https://www.data.jma.go.jp/obd/stats/etrn/view/{type}_{pattern}.php?prec_no={self.pre_id}&block_no={self.s_id}&year={yyyy}&month={mm}&day={dd}&view=p1'
          out = search_data(url)
          # 日付の列を追加（24個分）
          date = pd.DataFrame((np.full([24,1], f'{yyyy}{mm}{dd}')),columns=['日付'])
          df = pd.DataFrame(out, columns=['時','現地気圧','海面気圧','降水量','気温','露点温度','蒸気圧','湿度','風速','風向','日照時間','全天日射量','降雪','積雪','天気','雲量','視程'])
          df = pd.concat([date, df],axis=1)
          data2 = pd.concat([data2, df])
          data2.to_csv(f'{self.a_name}_{self.st_name}_{self.start}-{self.end}_{type}.csv',index=None, encoding='SJIS')
        else:
          pattern = 'a1'
          url = f'https://www.data.jma.go.jp/obd/stats/etrn/view/{type}_{pattern}.php?prec_no={self.pre_id}&block_no={self.s_id}&year={yyyy}&month={mm}&day={dd}&view=p1'
          out = search_data(url)
          date = pd.DataFrame((np.full([24,1], f'{yyyy}{mm}{dd}')),columns=['日付'])
          df = pd.DataFrame(out, columns=['時','降水量','気温','風速','風向','日照時間','降雪','積雪'])
          df = pd.concat([date, df],axis=1)
          data2_ = pd.concat([data2_, df])
          data2_.to_csv(f'{self.a_name}_{self.st_name}_{self.start}-{self.end}_{type}.csv',index=None, encoding='SJIS')

      elif type=='10min':
        if len(self.s_id) == 5:
          pattern = 's1'
          url = f'https://www.data.jma.go.jp/obd/stats/etrn/view/{type}_{pattern}.php?prec_no={self.pre_id}&block_no={self.s_id}&year={yyyy}&month={mm}&day={dd}&view=p1'
          out = search_data(url)
          # 日付の列を追加（6×24個分）
          date = pd.DataFrame((np.full([144,1], f'{yyyy}{mm}{dd}')),columns=['日付'])
          df = pd.DataFrame(out, columns=['時分','現地気圧','海面気圧','降水量','気温','相対湿度','平均風速','平均風向','最大瞬間風速','最大瞬間風向','日照時間'])
          df = pd.concat([date, df],axis=1)
          data3 = pd.concat([data3, df])
          data3.to_csv(f'{self.a_name}_{self.st_name}_{self.start}-{self.end}_{type}.csv',index=None, encoding='SJIS')
        else:
          pattern = 'a1'
          url = f'https://www.data.jma.go.jp/obd/stats/etrn/view/{type}_{pattern}.php?prec_no={self.pre_id}&block_no={self.s_id}&year={yyyy}&month={mm}&day={dd}&view=p1'
          out = search_data(url)
          date = pd.DataFrame((np.full([144,1], f'{yyyy}{mm}{dd}')),columns=['日付'])
          df = pd.DataFrame(out, columns=['時分','降水量','気温','平均風速','平均風向','最大瞬間風速','最大瞬間風向','日照時間'])
          df = pd.concat([date, df],axis=1)
          data3_ = pd.concat([data3_, df])
          data3_.to_csv(f'{self.a_name}_{self.st_name}_{self.start}-{self.end}_{type}.csv',index=None, encoding='SJIS')

      print(f'{self.a_name}_{self.st_name}_{yyyy}-{mm}-{dd}_{type}')
    print(f'{self.a_name}_{self.st_name}における{self.start}〜{self.end}の{type}データをダウンロードしました。')

ダウンロードしたい地域を入力してください： 東京都
ダウンロードしたい地点を入力してください： 東京
時間スケールを選択してください（daily or hourly or 10min）： daily
取得したい開始月(yyyymm)を入力してください： 201904
取得したい終了月(yyyymm)を入力してください： 201907


東京都_東京_2019-04-01_daily
東京都_東京_2019-05-01_daily
東京都_東京_2019-06-01_daily
東京都_東京_2019-07-01_daily
東京都_東京における201904〜201907のdailyデータをダウンロードしました。


In [ ]:
a_name = input('ダウンロードしたい地域を入力してください(例 東京都)：')
st_name = input('ダウンロードしたい地点を入力してください(例 東京)：')
amedas = Get_amedas_data(a_name,st_name)
type = input('時間スケールを選択してください（daily or hourly or 10min）：')
if type == 'daily':
    start= input('取得したい開始月(yyyymm)を入力してください：')
    end = input('取得したい終了月(yyyymm)を入力してください：')
    amedas.set_date1(start,end)
else:
    start= input('取得したい開始日(yyyymmddを入力してください：')
    end = input('取得したい終了日(yyyymmdd)を入力してください：')
    amedas.set_date2(start,end)

amedas.dl_data(type)

In [7]:
pd.read_csv("東京都_東京_201904-201907_daily.csv", encoding="shift-jis")

,年月,日,平均現地気圧,平均海面気圧,日降水量,最大1時間降水量,最大10分間降水量,平均気温,最高気温,最低気温,...,平均風速,最大風速,最大風向,最大瞬間風速,最大瞬間風向,日照時間,降雪,最深積雪,天気概況（昼）,天気概況（夜）
0,201904,1,1008.9,1011.9,5.5,4.0,2.0,8.8,16.2,4.2,...,3.1,7.4,西南西,12.0,南西,6.2,--,--,晴後曇,雨一時曇後晴
1,201904,2,1011.5,1014.5,0.0,0.5,0.0,7.2,13.1,2.3,...,3.1,6.2,南南東,11.2,北西,7.2,--,--,晴後曇,晴
2,201904,3,1014.3,1017.3,--,--,--,8.1,14.1,2.7,...,3.8,7.6,北西,14.0,北西,12.0,--,--,快晴,晴後一時薄曇
3,201904,4,1010.8,1013.8,--,--,--,10.6,16.7,3.4,...,2.8,6.4,南東,10.1,南東,11.7,--,--,快晴,晴後一時薄曇
4,201904,5,1001.4,1004.3,--,--,--,15.4,22.5,6.7,...,4.0,9.1,南,17.1,南南東,11.6,--,--,薄曇後晴,晴一時薄曇
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,201907,27,1007.5,1010.2,4.0,3.5,2.0,28.0,31.4,25.1,...,5.4,8.4,南南東,15.2,南南西,4.8,--,--,晴一時曇,曇後雨
118,201907,28,1009.0,1011.7,15.0,5.0,2.0,27.7,32.3,25.0,...,3.6,8.0,南,14.6,南,5.6,--,--,晴時々曇一時雨,晴時々薄曇
119,201907,29,1009.4,1012.1,0.0,0.0,0.0,29.0,33.7,25.3,...,2.6,6.0,南東,9.1,南東,9.2,--,--,晴一時曇,曇時々晴
120,201907,30,1008.0,1010.7,--,--,--,29.5,33.6,26.0,...,3.0,6.6,南南東,10.6,南南東,10.5,--,--,晴後一時薄曇,晴一時薄曇
